In [78]:
import pandas as pd
import fasttext
from logger import logger

import multiprocessing
import glob
from tqdm import tqdm
import xml.etree.ElementTree as ET
from itertools import chain
import numpy as np
import csv

In [4]:
logger.info('test')

2023-03-11 19:07:19,981 - test


# Level 1: Classifying Product Names to Categories

## Data preparation

In [ ]:
# !python createContentTrainingData.py --output /workspace/datasets/fasttext/labeled_products.txt

In [ ]:
# !head -1 /workspace/datasets/fasttext/labeled_products.txt

In [ ]:
# !tail -1 /workspace/datasets/fasttext/labeled_products.txt

In [ ]:
# Runs fine in command line
## !shuf /workspace/datasets/fasttext/labeled_products.txt --random-source=<(seq 99999) > /workspace/datasets/fasttext/shuffled_labeled_products.txt

In [ ]:
# !head -10000 /workspace/datasets/fasttext/shuffled_labeled_products.txt > /workspace/datasets/fasttext/training_data.txt
# !tail -10000 /workspace/datasets/fasttext/shuffled_labeled_products.txt > /workspace/datasets/fasttext/test_data.txt

In [ ]:
# !wc /workspace/datasets/fasttext/training_data.txt

In [ ]:
# !wc /workspace/datasets/fasttext/test_data.txt

## Training default model

In [4]:
product_classifier = fasttext.train_supervised(input='/workspace/datasets/fasttext/training_data.txt')

Read 0M words
Number of words:  10754
Number of labels: 1341
Progress: 100.0% words/sec/thread:    7868 lr:  0.000000 avg.loss: 13.822425 ETA:   0h 0m 0s


In [5]:
product_classifier.predict('Sharp - 46" 1080p Flat-Panel LCD HDTV')

(('__label__abcat0101001',), array([0.56883866]))

In [6]:
product_classifier.predict('Nikon D3000 10.2MP Digital SLR with Extra 55-200mm Lens, Tripod and Bag')

(('__label__pcmcat180400050006',), array([0.32290113]))

In [7]:
product_classifier.predict('Sony - VAIO Laptop with Intel® Centrino® Processor Technology - Sangria Red')

(('__label__pcmcat247400050000',), array([0.09211702]))

## Check labels

In [8]:
import xmltodict

In [9]:
# Parse xml into dictionary
with open('/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml') as fd:
    category = xmltodict.parse(fd.read())

In [10]:
cat_df = pd.DataFrame(category['categories']['category'])

In [11]:
cat_df[cat_df['id'] == 'abcat0101001']

,id,name,path,subCategories
28,abcat0101001,All Flat-Panel TVs,"{'category': [{'id': 'cat00000', 'name': 'Best...","{'category': [{'id': 'pcmcat138100050024', 'na..."


In [12]:
cat_df[cat_df['id'] == 'pcmcat180400050006']

,id,name,path,subCategories
2551,pcmcat180400050006,DSLR Package Deals,"{'category': [{'id': 'cat00000', 'name': 'Best...",None


In [13]:
cat_df[cat_df['id'] == 'pcmcat247400050000']

,id,name,path,subCategories
3955,pcmcat247400050000,PC Laptops,"{'category': [{'id': 'cat00000', 'name': 'Best...",None


## Evaluate default model

In [16]:
k = 1
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 17:31:38,923 - Evaluation on 9,669 samples - precision@1: 0.1667, recall@1: 0.1667


In [17]:
k = 5
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 17:31:50,665 - Evaluation on 9,669 samples - precision@5: 0.0540, recall@5: 0.2701


In [19]:
k = 10
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 17:32:07,994 - Evaluation on 9,669 samples - precision@10: 0.0333, recall@10: 0.3327


## Update params

In [20]:
product_classifier = fasttext.train_supervised(input='/workspace/datasets/fasttext/training_data.txt', 
                                               lr=1.0, wordNgrams=2, epoch=25)

Read 0M words
Number of words:  10754
Number of labels: 1341
Progress: 100.0% words/sec/thread:   15804 lr:  0.000000 avg.loss:  1.249754 ETA:   0h 0m 0s


In [21]:
k = 1
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 17:33:17,233 - Evaluation on 9,669 samples - precision@1: 0.6540, recall@1: 0.6540


## Data preparation
- Remove all non-alphanumeric chars other than underscore
- Lowercase all chars
- Trip excess space chars

In [22]:
!cat /workspace/datasets/fasttext/shuffled_labeled_products.txt |sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]_]/ /g" | tr -s ' ' > /workspace/datasets/fasttext/normalized_labeled_products.txt

In [24]:
!head -5 /workspace/datasets/fasttext/shuffled_labeled_products.txt

__label__abcat0401002 Canon - PowerShot 10.0-Megapixel Digital Camera - Black
__label__abcat0701001 Microsoft - Xbox 360 4GB Special Edition Kinect Family Bundle
__label__abcat0905001 KitchenAid - Architect P Series 23-7/8" Tall Tub Built-in Dishwasher - Black
__label__abcat0106008 Bush - Midnight Mist Audio Tower
__label__pcmcat209000050008 Viewsonic - ViewPad V7E_1WNA1US7_01 7" 4 GB Tablet Computer - Wi-Fi - Samsung S5PV210 1 GHz - White


In [23]:
!head -5 /workspace/datasets/fasttext/normalized_labeled_products.txt

__label__abcat0401002 canon powershot 10 0 megapixel digital camera black
__label__abcat0701001 microsoft xbox 360 4gb special edition kinect family bundle
__label__abcat0905001 kitchenaid architect p series 23 7 8 tall tub built in dishwasher black
__label__abcat0106008 bush midnight mist audio tower
__label__pcmcat209000050008 viewsonic viewpad v7e_1wna1us7_01 7 4 gb tablet computer wi fi samsung s5pv210 1 ghz white


In [25]:
!head -10000 /workspace/datasets/fasttext/normalized_labeled_products.txt > /workspace/datasets/fasttext/training_data.txt
!tail -10000 /workspace/datasets/fasttext/normalized_labeled_products.txt > /workspace/datasets/fasttext/test_data.txt

In [26]:
product_classifier = fasttext.train_supervised(input='/workspace/datasets/fasttext/training_data.txt', 
                                               lr=1.0, wordNgrams=2, epoch=25)

Read 0M words
Number of words:  8449
Number of labels: 1341
Progress: 100.0% words/sec/thread:    9917 lr:  0.000000 avg.loss:  1.166470 ETA:   0h 0m 0s 65.8% words/sec/thread:    8066 lr:  0.342373 avg.loss:  1.770161 ETA:   0h 0m 7s


In [27]:
k = 1
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 17:36:43,262 - Evaluation on 9,669 samples - precision@1: 0.6521, recall@1: 0.6521


## Filtering on category

In [53]:
files = glob.glob(f'/workspace/datasets/product_data/products/*.xml')
min_products = 2

In [42]:
names_as_labels = False

def transform_name(product_name):
    # IMPLEMENT
    return product_name

def _label_filename(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    labels = []
    for child in root:
        # Check to make sure category name is valid and not in music or movies
        if (child.find('name') is not None and child.find('name').text is not None and
            child.find('categoryPath') is not None and len(child.find('categoryPath')) > 0 and
            child.find('categoryPath')[len(child.find('categoryPath')) - 1][0].text is not None and
            child.find('categoryPath')[0][0].text == 'cat00000' and
            child.find('categoryPath')[1][0].text != 'abcat0600000'):
              # Choose last element in categoryPath as the leaf categoryId or name
              if names_as_labels:
                  cat = child.find('categoryPath')[len(child.find('categoryPath')) - 1][1].text.replace(' ', '_')
              else:
                  cat = child.find('categoryPath')[len(child.find('categoryPath')) - 1][0].text
              # Replace newline chars with spaces so fastText doesn't complain
              name = child.find('name').text.replace('\n', ' ')
              labels.append((cat, transform_name(name)))
    return labels

In [90]:
all_labels = tqdm(multiprocessing.Pool().imap(_label_filename, files[:3]), total=3)


  0%|                                                                           | 0/3 [00:00<?, ?it/s]

In [91]:
all_labels = [label for label in all_labels]


100%|███████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.09s/it]


In [100]:
all_labels = list(chain.from_iterable(all_labels))

In [101]:
label_df = pd.DataFrame(all_labels, columns=['cat', 'name'])

In [60]:
label_df = pd.DataFrame(all_labels, columns=['cat', 'name'])

In [61]:
label_agg = label_df.groupby('cat').agg({'name': 'count'}).reset_index()

In [62]:
filtered_categories = set(label_agg.loc[label_agg['name'] > 2, 'cat'].to_list())

In [64]:
label_df[label_df['cat'].isin(filtered_categories)]

,cat,name
7,abcat0811012,AT&T - 3.6V NiCad Battery for 900MHz Phones
13,pcmcat165900050033,Metra - Radio Installation Dash Kit for Most 1...
14,pcmcat165900050033,Metra - Radio Dash Multikit for Select GM Vehi...
15,pcmcat165900050034,Metra - Wiring Harness for Select 1998-2008 Fo...
16,pcmcat165900050034,Metra - Turbo Wire Aftermarket Radio Wire Harn...
17,pcmcat165900050034,Metra - Wiring Harness for Most 1986-1998 Hond...
21,pcmcat165900050033,Best Buy - Mazda Multi In-Dash Installation Kit
23,abcat0811012,Energizer - 3.6V NiCad Battery for 900MHz Phones
24,abcat0811012,Energizer - 3.6V NiCad Battery for 900MHz Phones
25,abcat0811012,Energizer - 3.6V NiCad Battery for 900MHz Phones


In [104]:
!python createContentTrainingData.py --output /workspace/datasets/fasttext/pruned_labeled_products.txt --min_products 500

Writing results to /workspace/datasets/fasttext/pruned_labeled_products.txt
100%|█████████████████████████████████████████| 256/256 [01:22<00:00,  3.12it/s]
Category count: 32, Product count: 28,921


In [111]:
# !shuf /workspace/datasets/fasttext/pruned_labeled_products.txt --random-source=<(seq 99999) > /workspace/datasets/fasttext/shuffled_pruned_labeled_products.txt

/usr/bin/sh: 1: Syntax error: "(" unexpected


In [112]:
!cat /workspace/datasets/fasttext/shuffled_pruned_labeled_products.txt |sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]_]/ /g" | tr -s ' ' > /workspace/datasets/fasttext/normalized_pruned_labeled_products.txt

In [113]:
!head -10000 /workspace/datasets/fasttext/normalized_pruned_labeled_products.txt > /workspace/datasets/fasttext/training_data.txt
!tail -10000 /workspace/datasets/fasttext/normalized_pruned_labeled_products.txt > /workspace/datasets/fasttext/test_data.txt

In [114]:
product_classifier = fasttext.train_supervised(input='/workspace/datasets/fasttext/training_data.txt', 
                                               lr=1.0, epoch=25)

Read 0M words
Number of words:  5373
Number of labels: 29
Progress: 100.0% words/sec/thread:  168711 lr:  0.000000 avg.loss:  0.042553 ETA:   0h 0m 0s


In [115]:
k = 1
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 18:30:54,304 - Evaluation on 8,473 samples - precision@1: 0.9629, recall@1: 0.9629


In [116]:
product_classifier = fasttext.train_supervised(input='/workspace/datasets/fasttext/training_data.txt', 
                                               lr=1.0, epoch=25, wordNgrams=2)

Read 0M words
Number of words:  5373
Number of labels: 29
Progress: 100.0% words/sec/thread:  146558 lr:  0.000000 avg.loss:  0.037745 ETA:   0h 0m 0s


In [117]:
k = 1
n, prec, rec = product_classifier.test(path='/workspace/datasets/fasttext/test_data.txt', k=k)
logger.info(f'Evaluation on {n:,} samples - precision@{k}: {prec:.4f}, recall@{k}: {rec:.4f}')

2023-03-11 18:31:14,246 - Evaluation on 8,473 samples - precision@1: 0.9645, recall@1: 0.9645


# Level 2: Derive Synonyms from Content

## Data preparation

In [119]:
!python createContentTrainingData.py --output /workspace/datasets/fasttext/products.txt --label name

Writing results to /workspace/datasets/fasttext/products.txt
100%|█████████████████████████████████████████| 256/256 [01:16<00:00,  3.35it/s]
Category count: 1803, Product count: 115,503


In [120]:
!cat /workspace/datasets/fasttext/products.txt |  cut -c 10- | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]]/ /g" | tr -s ' ' > /workspace/datasets/fasttext/normalized_products.txt

In [121]:
title_model = fasttext.train_unsupervised(input='/workspace/datasets/fasttext/normalized_products.txt',
                                          model='skipgram')

Read 1M words
Number of words:  9205
Number of labels: 0
Progress: 100.0% words/sec/thread:   46712 lr:  0.000000 avg.loss:  1.220938 ETA:   0h 0m 0s 57.6% words/sec/thread:   37299 lr:  0.021222 avg.loss:  1.263421 ETA:   0h 0m 5s


In [122]:
title_model.get_nearest_neighbors('inkjet')

[(0.8836662769317627, 'deskjet'),
 (0.8633587956428528, 'officejet'),
 (0.8344438672065735, 'b1h'),
 (0.8252926468849182, 'c5280'),
 (0.8137577772140503, 'laserjet'),
 (0.8085622191429138, 'pixma'),
 (0.8048376441001892, 'lexmark'),
 (0.8021136522293091, 'mfc'),
 (0.8000170588493347, 'printers'),
 (0.7972469329833984, 'mvm')]

In [123]:
title_model.get_nearest_neighbors('netbook')

[(0.9316118359565735, 'netbooks'),
 (0.9038827419281006, 'jetbook'),
 (0.8332692980766296, 'nextbook'),
 (0.7719100713729858, 'atom'),
 (0.7648991942405701, 'eee'),
 (0.7603505253791809, '1005hab'),
 (0.7213680148124695, 'slip'),
 (0.7157060503959656, 'n270'),
 (0.6964132189750671, 'np'),
 (0.6922696828842163, 'zenbook')]

In [5]:
title_model = fasttext.train_unsupervised(input='/workspace/datasets/fasttext/normalized_products.txt',
                                          model='skipgram', minCount=100)

Read 1M words
Number of words:  1556
Number of labels: 0
Progress: 100.0% words/sec/thread:   43684 lr:  0.000000 avg.loss:  1.295575 ETA:   0h 0m 0s


In [6]:
title_model.get_nearest_neighbors('inkjet')

[(0.8110601305961609, 'printers'),
 (0.7996888160705566, 'printer'),
 (0.7449747920036316, 'laserjet'),
 (0.7298192977905273, 'copier'),
 (0.7144044041633606, 'lexmark'),
 (0.7010620832443237, 'packard'),
 (0.6940978765487671, 'photosmart'),
 (0.677263081073761, 'brother'),
 (0.675620973110199, 'ink'),
 (0.6616724729537964, 'hewlett')]

In [7]:
title_model.get_nearest_neighbors('netbook')

[(0.9127790927886963, 'netbooks'),
 (0.7936349511146545, 'atom'),
 (0.7041299939155579, 'eee'),
 (0.6468859910964966, 'slip'),
 (0.6237702369689941, 'laptop'),
 (0.5247266292572021, 'laptops'),
 (0.5208449959754944, 'team'),
 (0.518611490726471, 'briefcase'),
 (0.5173459053039551, 'mouse'),
 (0.5157207250595093, 'inspiron')]

In [17]:
tokens = ['headphones', 'laptop', 'freezer', 'nintendo', 'whirlpool', 'kodak', 'ps2',
          'razer', 'stratocaster', 'holiday', 'plasma', 'leather']

for token in tokens:
    nn = title_model.get_nearest_neighbors(token)
    logger.info(f'Nearest neighbors for: {token}')
    [logger.info(n) for n in nn]

2023-03-11 19:12:30,722 - Nearest neighbors for: headphones
2023-03-11 19:12:30,723 - (0.8840067386627197, 'ear')
2023-03-11 19:12:30,723 - (0.8646281957626343, 'earbud')
2023-03-11 19:12:30,724 - (0.7090734243392944, 'bud')
2023-03-11 19:12:30,726 - (0.6759501099586487, 'noise')
2023-03-11 19:12:30,726 - (0.6747713685035706, 'sennheiser')
2023-03-11 19:12:30,727 - (0.649059534072876, 'over')
2023-03-11 19:12:30,727 - (0.6453553438186646, 'microphones')
2023-03-11 19:12:30,728 - (0.6345415115356445, 'gumy')
2023-03-11 19:12:30,728 - (0.6153662800788879, 'skullcandy')
2023-03-11 19:12:30,729 - (0.6142438650131226, 'earbuds')
2023-03-11 19:12:30,730 - Nearest neighbors for: laptop
2023-03-11 19:12:30,730 - (0.8026874661445618, 'laptops')
2023-03-11 19:12:30,731 - (0.746196985244751, 'notebook')
2023-03-11 19:12:30,731 - (0.6591004729270935, 'notebooks')
2023-03-11 19:12:30,732 - (0.623769998550415, 'netbook')
2023-03-11 19:12:30,732 - (0.6076312065124512, 'macbook')
2023-03-11 19:12:30,7

In [18]:
title_model = fasttext.train_unsupervised(input='/workspace/datasets/fasttext/normalized_products.txt',
                                          model='skipgram', minCount=100, epoch=25)

Read 1M words
Number of words:  1556
Number of labels: 0
Progress: 100.0% words/sec/thread:   81133 lr:  0.000000 avg.loss:  1.225277 ETA:   0h 0m 0s words/sec/thread:   68192 lr:  0.025533 avg.loss:  1.259784 ETA:   0h 0m17s


In [21]:
results = title_model.get_nearest_neighbors('iphone')
[result for result in results if result[0] >0.70]

[(0.8484390377998352, '4s'),
 (0.7798793315887451, 'apple'),
 (0.7289857864379883, 'ipod')]

## Level 3: Integrating Synonyms with Search

In [23]:
# !cat /workspace/datasets/fasttext/normalized_products.txt | tr " " "\n" | grep "...." | sort | uniq -c | sort -nr | head -1000 | grep -oE '[^ ]+$' > /workspace/datasets/fasttext/top_words.txt

In [24]:
!head -10 /workspace/datasets/fasttext/top_words.txt

black
with
digital
white
case
memory
camera
electric
games
laptop


In [173]:
!wc /workspace/datasets/fasttext/top_words.txt

1000 1000 7441 /workspace/datasets/fasttext/top_words.txt


In [31]:
top_words = pd.read_csv('/workspace/datasets/fasttext/top_words.txt', names=['token'])

In [176]:
# minCount = 7 as nespresso only appears 8 times
title_model = fasttext.train_unsupervised(input='/workspace/datasets/fasttext/normalized_products.txt',
                                          model='skipgram', minCount=7, epoch=25)

Read 1M words
Number of words:  7664
Number of labels: 0
Progress: 100.0% words/sec/thread:   62352 lr:  0.000000 avg.loss:  1.001207 ETA:   0h 0m 0s lr:  0.040688 avg.loss:  1.166722 ETA:   0h 0m47s


In [177]:
def get_nn(token, threshold=0.75):
    results = title_model.get_nearest_neighbors(token)
    results = ','.join([result[1] for result in results if result[0] > threshold])
    return results

In [178]:
top_words['neighbors'] = top_words['token'].apply(get_nn)

In [179]:
top_words['output'] = top_words['token'] + ',' + top_words['neighbors']

In [180]:
top_words.tail(5)

,token,neighbors,output
995,georgia,"bulldogs,hokies,hawkeyes,virginia,longhorns","georgia,bulldogs,hokies,hawkeyes,virginia,long..."
996,vented,"externally,hoods","vented,externally,hoods"
997,toss,"tailgate,sales","toss,tailgate,sales"
998,organization,pedestals,"organization,pedestals"
999,grand,,"grand,"


In [181]:
top_words[top_words['token'] == 'cappuccino']

,token,neighbors,output
856,cappuccino,"nespresso,espresso,bialetti,gaggia,espressione","cappuccino,nespresso,espresso,bialetti,gaggia,..."


In [112]:
# top_words['output'].to_csv('/workspace/datasets/fasttext/synonyms.csv', index=False, header=False, quoting=3, quotechar='') #escapechar='|')

In [182]:
with open('/workspace/datasets/fasttext/synonyms.csv', 'w') as output:
    for i, row in top_words.iterrows():
        output.write(f'{row["output"]}\n')

In [183]:
!head -5 /workspace/datasets/fasttext/synonyms.csv

black,
with,
digital,
white,
case,cases


In [184]:
!tail -10 /workspace/datasets/fasttext/synonyms.csv

picks,pick,351,celluloid,tortoise
laundry,washer,pedestals
landscape,
celestron,telescopes,scopes,microscopes
salamander,designs,elba,barcelona
georgia,bulldogs,hokies,hawkeyes,virginia,longhorns
vented,externally,hoods
toss,tailgate,sales
organization,pedestals
grand,


In [185]:
!docker cp /workspace/datasets/fasttext/synonyms.csv opensearch-node1:/usr/share/opensearch/config/synonyms.csv

In [188]:
!python utilities/query.py --query earbuds

{
  "took": 7,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1205,
      "relation": "eq"
    },
    "max_score": 843.09705,
    "hits": [
      {
        "_index": "bbuy_products",
        "_id": "9084206",
        "_score": 843.09705,
        "_source": {
          "name": [
            "Apple\u00ae - Earbuds for Select Apple\u00ae iPod\u00ae Models"
          ],
          "shortDescription": [
            "Compatible with 4th-generation iPod nano, 120GB iPod classic and 2nd-generation iPod touch; remote for controlling music and video playback; microphone for recording voice memos"
          ]
        }
      },
      {
        "_index": "bbuy_products",
        "_id": "2197043",
        "_score": 121.19351,
        "_source": {
          "name": [
            "Sony - Earbud Headphones - Black"
          ],
          "shortDescription": [
            "13.5mm drivers; neodymiu

In [189]:
!python utilities/query.py --query earbuds --synonyms

Querying via synonyms
{
  "took": 9,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 4606,
      "relation": "eq"
    },
    "max_score": 851.03107,
    "hits": [
      {
        "_index": "bbuy_products",
        "_id": "9084206",
        "_score": 851.03107,
        "_source": {
          "name": [
            "Apple\u00ae - Earbuds for Select Apple\u00ae iPod\u00ae Models"
          ],
          "shortDescription": [
            "Compatible with 4th-generation iPod nano, 120GB iPod classic and 2nd-generation iPod touch; remote for controlling music and video playback; microphone for recording voice memos"
          ]
        }
      },
      {
        "_index": "bbuy_products",
        "_id": "3109302",
        "_score": 188.7468,
        "_source": {
          "name": [
            "Apple\u00ae - iPod touch\u00ae 8GB* MP3 Player (4th Generation - Latest Model) - White"
       

In [190]:
!python utilities/query.py --query nespresso

{
  "took": 4,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 8,
      "relation": "eq"
    },
    "max_score": 0.07420327,
    "hits": [
      {
        "_index": "bbuy_products",
        "_id": "1720059",
        "_score": 0.07420327,
        "_source": {
          "name": [
            "Nespresso - Essenza Espresso Maker - Black"
          ],
          "shortDescription": [
            "Automatic and programmable; compact brewing unit technology; included Aeroccino milk frother; backlit on/off and coffee volume buttons"
          ]
        }
      },
      {
        "_index": "bbuy_products",
        "_id": "3990516",
        "_score": 0.07420327,
        "_source": {
          "name": [
            "Nespresso - Essenza Espresso Maker - Black"
          ],
          "shortDescription": [
            "Makes espresso and lungo; 19-bar high-pressure pump; Thermobloc heating eleme

In [191]:
!python utilities/query.py --query nespresso --synonyms

Querying via synonyms
{
  "took": 6,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 270,
      "relation": "eq"
    },
    "max_score": 6.1888156,
    "hits": [
      {
        "_index": "bbuy_products",
        "_id": "2473053",
        "_score": 6.1888156,
        "_source": {
          "name": [
            "Dynex\u2122 - Espresso Basic Stand for TVs Up to 32\""
          ],
          "shortDescription": [
            "Accommodates up to 3 full-size components; additional storage for game consoles; 1 adjustable shelf; particle board; slide-on moldings; wire management"
          ]
        }
      },
      {
        "_index": "bbuy_products",
        "_id": "1720059",
        "_score": 0.074758835,
        "_source": {
          "name": [
            "Nespresso - Essenza Espresso Maker - Black"
          ],
          "shortDescription": [
            "Automatic and programmable

In [192]:
!python utilities/query.py --query dslr

{
  "took": 7,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 2837,
      "relation": "eq"
    },
    "max_score": 408.17416,
    "hits": [
      {
        "_index": "bbuy_products",
        "_id": "1980124",
        "_score": 408.17416,
        "_source": {
          "name": [
            "Canon - EOS Rebel T3i 18.0-Megapixel DSLR Camera with 18-55mm Lens - Black"
          ],
          "shortDescription": [
            "Vari-angle 3.0-inch Clear View LCD screen1080 full HD video3.7 FPS (frames per second)ISO 100-6400 (expandable to 12800)"
          ]
        }
      },
      {
        "_index": "bbuy_products",
        "_id": "1989073",
        "_score": 347.9671,
        "_source": {
          "name": [
            "Nikon - D5100 16.2-Megapixel DSLR Camera with 18-55mm VR Lens - Black"
          ],
          "shortDescription": [
            "High Resolution 16.2 MP DX-format

In [193]:
!python utilities/query.py --query dslr --synonyms

Querying via synonyms
{
  "took": 20,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 7023,
      "relation": "eq"
    },
    "max_score": 440.97403,
    "hits": [
      {
        "_index": "bbuy_products",
        "_id": "1980124",
        "_score": 440.97403,
        "_source": {
          "name": [
            "Canon - EOS Rebel T3i 18.0-Megapixel DSLR Camera with 18-55mm Lens - Black"
          ],
          "shortDescription": [
            "Vari-angle 3.0-inch Clear View LCD screen1080 full HD video3.7 FPS (frames per second)ISO 100-6400 (expandable to 12800)"
          ]
        }
      },
      {
        "_index": "bbuy_products",
        "_id": "1989073",
        "_score": 371.35364,
        "_source": {
          "name": [
            "Nikon - D5100 16.2-Megapixel DSLR Camera with 18-55mm VR Lens - Black"
          ],
          "shortDescription": [
            "High Reso